# Hands-on session: Machine Learning & Data Visualization
---
## 3 - Evaluating ML models
---


How do I run this?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/OHBM-Brainhack-2021)

Gitpod?

Binder?

---

## R2 score

---
## Mean Absolute Error

---
## Mean Squared Error

---
## Explained variance

---
## Pearson's correlation